In [2]:
import logging

import ee

logger = logging.getLogger('glossis-currents')
logging.basicConfig(level=logging.DEBUG)


In [3]:
def getWGS84Geometry():
    geometry = ee.Geometry.Polygon([
            [180,90],
            [0,90],
            [-180,90],
            [-180,-90],
            [0,-90],
            [180,-90],
            [180,90]
        ], 
        'EPSG:4326', 
        False
    )
    return geometry

def last(images):
    sorted = images.sort('system:time_start', False)
    last = sorted.first()
    return last

def tail(images, n):
    sorted = images.limit(n, 'system:time_start', False)
    return sorted

In [4]:
def exportCurrents(currents):
    """Extract the timestamp for unique variable names  (assumes new instances are overwritten)"""
    timeStamp = currents.date().format("yyyyMMddHHmm")
    currents = currents.set('time_stamp', timeStamp)
    flowmap = currents.unitScale(-0.5, 0.5)

    # First deal with the mask
    # Create unmask and replace  mask by missing value
    flowmap = flowmap.unmask(-9999)

    # take  the mask
    mask = flowmap.eq(-9999).select('b1')
    # Create a smooth mask
    mask = mask.float()

    mask = (
      mask
        .resample('bilinear')
        .where(land.mask(), 1)
        .unmask()
    )  

    # convert to 0,1
    flowmap = flowmap.clamp(0, 1)
    flowmap = flowmap.resample('bilinear')

    flowmap = (
      flowmap
        .convolve(ee.Kernel.gaussian(30000, 20000, 'meters'))
    )

    flowmap = flowmap.addBands(mask)
    flowmap = flowmap.rename(['u', 'v', 'mask'])
    # Convert three  channels to rgb
    flowmapRgb = flowmap.visualize()
    flowmapRgb = ee.Image(
        flowmapRgb
          .copyProperties(currents)
    )
    return flowmapRgb

In [6]:
def exportTimeStamp(timeStamp,  region):
    """This uses the global flowmapRgbs"""
    exportFilename ='glossis-current' + '-' + timeStamp
    selected  = flowmapRgbs.filter(ee.Filter.eq('time_stamp', timeStamp))
    flowmapRgb = selected.first()
    kwargs = {
        "image": flowmapRgb, 
        "description": exportFilename + '-flowmap', 
        "bucket": 'deltares-video-map', 
        "fileNamePrefix": exportFilename, 
        "dimensions": '8192x5760', 
        "region": region, 
        "crs": 'EPSG:4326'
    }
    logger.debug('exporting task {}'.format(kwargs))
    return ee.batch.Export.image.toCloudStorage(**kwargs)


In [7]:
ee.Initialize()
glossis = ee.ImageCollection("projects/dgds-gee/glossis/currents")
land = ee.Image("users/gena/land_polygons_image")

INFO:googleapiclient.discovery:URL being requested: GET https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/algorithms?prettyPrint=false&alt=json
DEBUG:google_auth_httplib2:Making request: POST https://accounts.google.com/o/oauth2/token


In [12]:
region = getWGS84Geometry()
currentImages =  tail(glossis,  20)

# Compute  all flowmaps
flowmapRgbs = currentImages.map(exportCurrents)

# Extract all time_stamps
timeStamps = flowmapRgbs.aggregate_array('time_stamp').getInfo()
# For each timestamp,  export the map
tasks = []
for  timeStamp in timeStamps:
    tasks.append(exportTimeStamp(timeStamp, region))


INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
DEBUG:glossis-currents:exporting task {'image': <ee.image.Image object at 0x112b94320>, 'description': 'glossis-current-202003270000-flowmap', 'bucket': 'deltares-video-map', 'fileNamePrefix': 'glossis-current-202003270000', 'dimensions': '8192x5760', 'region': <ee.geometry.Geometry object at 0x112c14128>, 'crs': 'EPSG:4326'}
DEBUG:glossis-currents:exporting task {'image': <ee.image.Image object at 0x112b1ac18>, 'description': 'glossis-current-202003261200-flowmap', 'bucket': 'deltares-video-map', 'fileNamePrefix': 'glossis-current-202003261200', 'dimensions': '8192x5760', 'region': <ee.geometry.Geometry object at 0x112c14128>, 'crs': 'EPSG:4326'}
DEBUG:glossis-currents:exporting task {'image': <ee.image.Image object at 0x112b6ae48>, 'description': 'glossis-current-202003260600-flowmap', 'bucket': 'deltares-video-map', 

[<Task EXPORT_IMAGE: glossis-current-202003270000-flowmap (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: glossis-current-202003261200-flowmap (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: glossis-current-202003260600-flowmap (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: glossis-current-202003260000-flowmap (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: glossis-current-202003251800-flowmap (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: glossis-current-202003251200-flowmap (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: glossis-current-202003250600-flowmap (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: glossis-current-202003250000-flowmap (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: glossis-current-202003241800-flowmap (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: glossis-current-202003241200-flowmap (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: glossis-current-202003240600-flowmap (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: glossis-current-202003240000-flowmap (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: glossis-current-202003231800-flowmap (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: glossis-current-2

In [9]:
# If you want  to debug,  you might want to try it on 1 image
if True:
    currents = currentImages.first()
    flowmapRgb = exportCurrents(currents)
    timeStamp = flowmapRgb.get('time_stamp').getInfo()
    task = exportTimeStamp(timeStamp, region)


INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
DEBUG:glossis-currents:exporting task {'image': <ee.image.Image object at 0x112bfee48>, 'description': 'glossis-current-202003270000-flowmap', 'bucket': 'deltares-video-map', 'fileNamePrefix': 'glossis-current-202003270000', 'dimensions': '8192x5760', 'region': <ee.geometry.Geometry object at 0x112b48438>, 'crs': 'EPSG:4326'}


In [13]:
for task in tasks:
    task.start()

INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/image:export?alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/image:export?alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/image:export?alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/image:export?alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/image:export?alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/image:export?alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [30]:
ee.batch.Task.list()

INFO:googleapiclient.discovery:URL being requested: GET https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/operations?pageSize=500&alt=json


[<Task EXPORT_IMAGE: glossis-current-202003220000-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003220600-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003221200-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003221800-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003230000-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003230600-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003231200-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003231800-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003240000-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003240600-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003241200-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003241800-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003250000-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003250600-flowmap (RUNNING)>,
 <Task EXPORT_IMAGE: glossis-current-202003251

In [31]:
task = tasks[-1]

In [25]:
ee.batch.Task.list()

INFO:googleapiclient.discovery:URL being requested: GET https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/operations?pageSize=500&alt=json


[<Task EXPORT_IMAGE: glossis-current-202003220000-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003220600-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003221200-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003221800-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003230000-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003230600-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003231200-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003231800-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003240000-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003240600-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003241200-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003241800-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003250000-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-202003250600-flowmap (READY)>,
 <Task EXPORT_IMAGE: glossis-current-20200325120

In [34]:
task.status()

INFO:googleapiclient.discovery:URL being requested: GET https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/operations/T3EMPLRCSZTOYV4DQUMPUYIE?alt=json


{'state': 'READY',
 'description': 'glossis-current-202003220000-flowmap',
 'creation_timestamp_ms': 1585307423228,
 'update_timestamp_ms': 1585307423228,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'T3EMPLRCSZTOYV4DQUMPUYIE',
 'name': 'projects/earthengine-legacy/operations/T3EMPLRCSZTOYV4DQUMPUYIE'}